In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import math

import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [60]:
def transfer(df):
    transferd = df
    
    transferd = transferd.drop(columns='Name')
    transferd = transferd.drop(columns='Ticket')
    
    # Age
    def fixAge(x): 
        if isinstance(x, float) and math.isnan(x):
            return -1
        else:
            return x
    def isInputedAge(x): 
        if isinstance(x, float) and math.isnan(x):
            return 0
        else:
            return 1
    transferd['AgeFixed'] = transferd['Age'].apply(fixAge)
    transferd['AgeInputed'] = transferd['Age'].apply(isInputedAge)
    transferd = transferd.drop(columns='Age')
    
    # Sex
    def transSex(x): 
        match x:
            case "male":
                return 1
            case _:
                return 0    
    transferd['SexCode'] = transferd['Sex'].apply(transSex)
    transferd = transferd.drop(columns='Sex')
    
    # Cabin
    def transCabin(x): 
        if isinstance(x, float) and math.isnan(x):
            return 0
        else:
            return 1
    transferd['CabinCode'] = transferd['Cabin'].apply(transCabin)
    transferd = transferd.drop(columns='Cabin')
    
    # Embarked
    def transEmbarked(x): 
        match x:
            case "C":
                return 1
            case "Q":
                return 2
            case "S":
                return 3
            case _:
                return 0    
    transferd['EmbarkedCode'] = transferd['Embarked'].apply(transEmbarked)
    transferd = transferd.drop(columns='Embarked')
    
    transferd = transferd.fillna({'Fare': 0})
              
    return transferd

# test
def transfer_test():
    df = pd.read_csv('/kaggle/input/titanic/train.csv')
    print(df.dtypes)
    print('# ================== test ==================')
    print(df)
    df = transfer(df)
    print('# ================== result ==================')
    print(df)
    print('## 欠損値を含む行を検索')
    print(df[df.isnull().any(axis=1)])
    
transfer_test()

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
# ================== test ==================
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs T

In [55]:
def read_train_file():    
    df = pd.read_csv('/kaggle/input/titanic/train.csv')
    df = transfer(df)
    return df

# test
def read_train_file_test():
    df = read_train_file()
    print(df)
    print('## 欠損値を含む行を検索')
    print(df[df.isnull().any(axis=1)])
read_train_file_test()

     PassengerId  Survived  Pclass  SibSp  Parch     Fare  AgeFixed  \
0              1         0       3      1      0   7.2500      22.0   
1              2         1       1      1      0  71.2833      38.0   
2              3         1       3      0      0   7.9250      26.0   
3              4         1       1      1      0  53.1000      35.0   
4              5         0       3      0      0   8.0500      35.0   
..           ...       ...     ...    ...    ...      ...       ...   
886          887         0       2      0      0  13.0000      27.0   
887          888         1       1      0      0  30.0000      19.0   
888          889         0       3      1      2  23.4500      -1.0   
889          890         1       1      0      0  30.0000      26.0   
890          891         0       3      0      0   7.7500      32.0   

     AgeInputed  SexCode  CabinCode  EmbarkedCode  
0             1        1          0             3  
1             1        0          1        

In [61]:
def read_test_file():    
    df = pd.read_csv('/kaggle/input/titanic/test.csv')
    df = transfer(df)
    return df

# test
def read_test_file_test():
    df = read_test_file()
    print(df)
    print('## 欠損値を含む行を検索')
    print(df[df.isnull().any(axis=1)])
read_test_file_test()

     PassengerId  Pclass  SibSp  Parch      Fare  AgeFixed  AgeInputed  \
0            892       3      0      0    7.8292      34.5           1   
1            893       3      1      0    7.0000      47.0           1   
2            894       2      0      0    9.6875      62.0           1   
3            895       3      0      0    8.6625      27.0           1   
4            896       3      1      1   12.2875      22.0           1   
..           ...     ...    ...    ...       ...       ...         ...   
413         1305       3      0      0    8.0500      -1.0           0   
414         1306       1      0      0  108.9000      39.0           1   
415         1307       3      0      0    7.2500      38.5           1   
416         1308       3      0      0    8.0500      -1.0           0   
417         1309       3      1      1   22.3583      -1.0           0   

     SexCode  CabinCode  EmbarkedCode  
0          1          0             2  
1          0          0        

In [35]:
def learning(df, features):
    
    X_train = df[features]
    y_train = df['Survived']
    
    model = RandomForestClassifier(random_state=1234)
    model.fit(X_train, y_train)
    print(model.feature_importances_)
    return model

In [52]:
def predict(model, df, features):

    X_test = df[features]
    y_result = model.predict(X_test)
    return y_result

In [44]:
def evaluate(model, df, features):

    X_test = df[features]
    y_test = df['Survived']
    score = model.score(X_test, y_test)
    print('score=' + str(score))
    return score

In [48]:
def create_model(features, X_train, X_test):
    
    
    # 学習
    model = learning(X_train, features)
    
    # 結果評価
    score = evaluate(model, X_test, features)
    
    return model


# test
def create_model_test():
    features = ['Pclass'
        ,'SibSp'
        ,'Parch'      
        ,'Fare'
        ,'AgeFixed'
        ,'AgeInputed'
        ,'SexCode'
        ,'CabinCode'
        ,'EmbarkedCode'
    ]
    
    df_all = read_train_file()
    X_train, X_test = train_test_split(df_all, test_size=0.3, random_state=1234)
    
    create_model(features, X_train, X_test)
    
create_model_test()

[0.08021275 0.04535919 0.04422268 0.2758403  0.21662872 0.01632395
 0.24595071 0.0383218  0.0371399 ]
score=0.8022388059701493


In [78]:
def save_result(df, y_result):
    result = df[['PassengerId']]
    result['Survived'] = y_result
    print(result)
    result.to_csv('/kaggle/working/titanic_20240104.csv', index=False)

In [79]:
def execute():
    features = ['Pclass'
        ,'SibSp'
        ,'Parch'      
        ,'Fare'
        ,'AgeFixed'
        ,'AgeInputed'
        ,'SexCode'
        ,'CabinCode'
        ,'EmbarkedCode'
    ]
    
    df_train = read_train_file()
    X_train, X_test = train_test_split(df_train, test_size=0.3, random_state=1234)
    
    model = create_model(features, X_train, X_test)
    
    df_test = read_test_file()
    y_result = predict(model, df_test, features)
    save_result(df_test, y_result)

execute()

[0.08021275 0.04535919 0.04422268 0.2758403  0.21662872 0.01632395
 0.24595071 0.0383218  0.0371399 ]
score=0.8022388059701493
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         1
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


/tmp/ipykernel_42/3380068702.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Survived'] = y_result
